In [13]:
import dgl
import torch
import torch.nn as nn
import numpy as np
import networkx as nx

In [14]:
def collate(samples):
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

In [15]:
from torch.utils.data import DataLoader

min_number_nodes = 32
max_number_nodes = 64

num_train_examples = 240
num_test_examples = 1000
minibatch_size = 16

train_set = dgl.data.MiniGCDataset(num_train_examples, min_number_nodes, max_number_nodes, seed=0)
test_set = dgl.data.MiniGCDataset(num_test_examples, min_number_nodes * 2, max_number_nodes * 2, seed=0)

train_dataloader = DataLoader(
    train_set, batch_size=minibatch_size, drop_last=False, shuffle=True, collate_fn=collate)
test_dataloader = DataLoader(
    test_set, batch_size=minibatch_size, drop_last=False, shuffle=True, collate_fn=collate)

In [16]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.conv3 = GraphConv(hidden_dim, int(hidden_dim / 2))
        self.conv4 = GraphConv(int(hidden_dim / 2), int(hidden_dim / 2))
        self.conv5 = GraphConv(int(hidden_dim / 2), int(hidden_dim / 4))
        self.classifier = nn.Linear(int(hidden_dim / 4), num_classes)

    def forward(self, graph):
        h = graph.in_degrees().view(-1, 1).float() # Use node degree as the initial node feature
        h = self.conv1(graph, h)
        h = nn.functional.relu(h)
        h = self.conv2(graph, h)
        h = nn.functional.relu(h)
        h = self.conv3(graph, h)
        h = nn.functional.relu(h)
        h = self.conv4(graph, h)
        h = nn.functional.relu(h)
        h = self.conv5(graph, h)
        h = nn.functional.relu(h)
        graph.ndata['h'] = h
        
        global_h = dgl.mean_nodes(graph, 'h')
        return self.classifier(global_h)

In [17]:
def train(graphs, model, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs+1):
        for batched_graph, labels in train_dataloader:
            pred = model(batched_graph)
            loss = nn.functional.cross_entropy(pred, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        num_correct = 0
        num_tests = 0
        for batched_graph, labels in test_dataloader:
            pred = model(batched_graph)
            num_correct += (pred.argmax(1) == labels).sum().item()
            num_tests += len(labels)

        if epoch % 5 == 0:
            print('In epoch {}, training loss: {:4f}, test accuracy: {}%'.format(
                epoch, loss, (num_correct / num_tests) * 100))

In [18]:
model = model = GCN(1, 256, train_set.num_classes)
train(train_set, model, 100)

In epoch 0, training loss: 1.739795, test accuracy: 12.5%
In epoch 5, training loss: 1.386608, test accuracy: 19.400000000000002%
In epoch 10, training loss: 0.931678, test accuracy: 61.199999999999996%
In epoch 15, training loss: 0.675700, test accuracy: 59.9%
In epoch 20, training loss: 0.747758, test accuracy: 65.2%
In epoch 25, training loss: 0.501721, test accuracy: 66.0%
In epoch 30, training loss: 0.442105, test accuracy: 68.89999999999999%
In epoch 35, training loss: 0.376641, test accuracy: 65.3%
In epoch 40, training loss: 0.325843, test accuracy: 71.5%
In epoch 45, training loss: 0.647940, test accuracy: 69.3%
In epoch 50, training loss: 0.325453, test accuracy: 68.5%
In epoch 55, training loss: 0.251541, test accuracy: 69.19999999999999%
In epoch 60, training loss: 0.401786, test accuracy: 73.7%
In epoch 65, training loss: 0.395171, test accuracy: 60.099999999999994%
In epoch 70, training loss: 0.840737, test accuracy: 68.60000000000001%
In epoch 75, training loss: 0.735209